In [10]:
# import dependencies
import numpy as np
import pandas as pd
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [11]:
import datetime as dt

In [12]:
from flask import Flask, jsonify

In [13]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [14]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [15]:
# Reflect an existing database into new model
Base = automap_base()
# Reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [16]:
# save the references to each table (shown above)
Measurement = Base.classes.measurement
Station = Base.classes.station

In [17]:
# Create link/session from Python to the database
session = Session(engine)

In [18]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [19]:
# Create an app
app = Flask(__name__)

In [20]:
# Create an app
app = Flask(__name__)

In [21]:
# home page
@app.route("/")
def home():
    print("Server received request for 'Home' page...")
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br>"
        f"/api/v1.0/stations<br>"
        f"/api/v1.0/tobs<br>"
        f"/api/v1.0/ + start date<br>"
        f"/api/v1.0/ + start date/ + end date"
    )

In [22]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    print("Server received request for 'precipitation' page...")
    last_date = session.query(func.max(Measurement.date)).all()
    last_date = last_date[0][0]
    last_date = dt.datetime.strptime(last_date, '%Y-%m-%d').date()
    year_before_date = ((last_date - relativedelta(years = 1)).strftime('%Y-%m-%d'))
    query = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_before_date).order_by(Measurement.date.desc()).all()
    data = {date: prcp for date, prcp in query}
    return jsonify(data)

In [23]:
@app.route("/api/v1.0/stations")
def stations():
    print("Server received request for 'stations' page...")
    query = session.query(Station.station).all()
    stations = list(np.ravel(query))
    return jsonify(stations)

@app.route("/api/v1.0/tobs")
def tobs():
    print("Server received request for 'tobs' page...")
    last_date = session.query(func.max(Measurement.date)).all()
    last_date = last_date[0][0]
    last_date = dt.datetime.strptime(last_date, '%Y-%m-%d').date()
    year_before_date = ((last_date - relativedelta(years = 1)).strftime('%Y-%m-%d'))
    query = session.query(Measurement.tobs).filter(Measurement.date >= year_before_date).order_by(Measurement.date.desc()).all()
    data = list(np.ravel(query))
    return jsonify(data)

In [24]:
@app.route("/api/v1.0/<start>")
def startDate(start):
    print("Server received request for 'start' page...")
    query = session.query(Measurement.tobs).filter(Measurement.date >= start).all()
    df = pd.DataFrame(query)
    tmin = df.min()
    tavg = df.mean()
    tmax = df.max()
    data = [tmin, tavg, tmax]
    data = list(np.ravel(data))
    return jsonify(data)

In [25]:
@app.route("/api/v1.0/<start>/<end>")
def startANDendDate(start, end):
    print("Server received request for 'start/end' page...")
    query = session.query(Measurement.tobs).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    df = pd.DataFrame(query) 
    tmin = df.min()
    tavg = df.mean()
    tmax = df.max()
    data = [tmin, tavg, tmax]
    data = list(np.ravel(data))
    return jsonify(data)

if __name__ == "__main__":
    app.run(debug=True)# 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use